# Goroutines

In [1]:
import(
    "time"
    "fmt"
)
func spinner(delay time.Duration){
    for {
        for _, r := range `-\|/`{
            fmt.Printf("\r%c", r)
            time.Sleep(delay)
        }
    }
}
func fib(x int) int {
    if x < 2{
        return x
    }
    return fib(x-1) + fib(x-2)
}
go spinner(100 * time.Millisecond)
n:=4
f := fib(n)
fmt.Printf("\rFibonacci(%d)=%d\n", n, f)

Fibonacci(4)=3


16 <nil>

In [4]:
import(
    "fmt"
    "sync"
    "time"
)
func makeThumb(filenames <-chan string) {
    sizes := make(chan int64)
    var wg sync.WaitGroup
    for f := range filenames { // 取出通道中所有数据，如果通道没有cloose，则阻塞，否则range结束
        wg.Add(1)
        go func(f string) {
            defer wg.Done()
            sizes <- int64(len(f))
            fmt.Printf("filename: %v, size: %v\n", f, len(f))
        }(f)
    }
    go func() {
        fmt.Printf("end\n")
        wg.Wait()
        close(sizes)
    }()
    var total int64
    for size := range sizes {
        total += size
        fmt.Printf("total: %v\n", total)
    }
    fmt.Printf("total: %v\n", total)
}
func makeFile() chan string{
    filenames := make(chan string, 2)
    fmt.Printf("make file star\n")
    filenames <- "sfx"
    filenames <- "songfuxing"
    fmt.Printf("make file done\n")
    close(filenames)
    return filenames
}
filenames := makeFile()
makeThumb(filenames)

make file star
make file done
end
filename: songfuxing, size: 10
total: 10
total: 13
filename: sfx, size: 3
total: 13


# Example: Concurrent Web Crawler

In [1]:
import(
    "fmt"
    "links"
    "log"
)
func crawl(url string) []string {
    fmt.Println(url)
    list, err := links.Extract(url)
    if err != nil {
        log.Print(err)
    }
    return list
}
crawl("www.baidu.com")

// warning: could not find package "github.com/gopherdata/gophernotes/imports" in $GOPATH = "/Users/songfuxing/go"
//          command 'import _b "path/to/some/package"' may not work correctly.



ERROR: error loading package "links" metadata: -: malformed module path "links": missing dot in first path element

In [2]:
package links
import(
    "fmt"
    "net/http"
    "golang.org/x/net/html"
)
func Extract(url string) ([]string, error) {
    resp, err := http.Get(url)
    if err != nil {
        return nil, err
    }
    if resp.StatusCode != http.StatusOK {
        resp.Body.Close()
        return nil, fmt.Errorf("getting %s: %s", url, resp.Status)
    }
    doc, err := html.Parse(resp.Body)
    resp.Body.Close()
    if err != nil {
        return nil, fmt.Errorf("parsing %s: as HTML: %v", url, err)
    }
    var links []string
    visitNode := func(n *html.Node) {
        if n.Type == html.ElementNode && n.Data == "a" {
            for _, a := range n.Attr {
                if a.key != "href" {
                    continue
                }
                link, err := resp.Request.URL.Parse(a.Val)
                if err != nil {
                    continue
                }
                links = append(links, link.String())
            }
        }
    }
    foreachNode(doc, visitNode, nil)
    return links, nil
}


ERROR: error loading package "golang.org/x/net/html" metadata: -: module golang.org/x/net/html: Get https://go-mod-proxy.byted.org/golang.org/x/net/html/@v/list: dial tcp: lookup go-mod-proxy.byted.org: no such host